# Building A Chatbot
We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

Conversational RAG: Enable a chatbot experience over an external source of data
Agents: Build a chatbot that can take actions

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key = os.getenv('GROQ_API_KEY')
groq_api_key


'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'

In [2]:
from langchain_groq import ChatGroq
model = ChatGroq(model='Gemma2-9b-It',groq_api_key=groq_api_key)

In [3]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, I am Preetham and I am a chief AI Engineer at Nvidia")])

AIMessage(content="Hello Preetham, it's nice to meet you!\n\nIt's impressive to know you're a Chief AI Engineer at Nvidia. That's a fascinating and impactful role. What are you currently working on that you're most excited about?  \n\nI'm eager to learn more about your work and the advancements happening in AI at Nvidia.\n", response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 29, 'total_tokens': 105, 'completion_time': 0.156022637, 'prompt_time': 0.002576525, 'queue_time': None, 'total_time': 0.158599162}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-a2b4c4ed-b9eb-4ab2-a8a6-f5bc85680dbc-0', usage_metadata={'input_tokens': 29, 'output_tokens': 76, 'total_tokens': 105})

In [4]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, I am Preetham and I am a chief AI Engineer at Nvidia"),
    AIMessage(content="It's nice to meet you, Preetham!\n\nThat's an impressive title. As a chief AI engineer at Nvidia, you must be at the forefront of some exciting developments in the field of artificial intelligence.\n\nWhat are some of the most interesting projects you're working on these days?  \n\nI'm eager to learn more about Nvidia's contributions to AI.\n"),
    HumanMessage(content="Hey What is my name and What do I do?") 
])

AIMessage(content="According to our conversation, your name is Preetham and you are a chief AI Engineer at Nvidia.  \n\nIs there anything else you'd like to know or discuss? 😊 \n", response_metadata={'token_usage': {'completion_tokens': 40, 'prompt_tokens': 129, 'total_tokens': 169, 'completion_time': 0.080987901, 'prompt_time': 0.005746697, 'queue_time': None, 'total_time': 0.086734598}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-84582549-b1d1-41d7-a1cb-185431400f24-0', usage_metadata={'input_tokens': 129, 'output_tokens': 40, 'total_tokens': 169})

Message History


We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [5]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}
#this func creates session id that distunguish one chat Id with another and it also retrieves chat history corresponding to session_id
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]
#to interact with the llm based on the chat history
with_message_history = RunnableWithMessageHistory(model,get_session_history)

In [6]:
config={"configurable":{"session_id":"chat1"}}

In [7]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Preetham and I am a Chief AI Engineer")],
    config=config
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [8]:
response.content

"Hello Preetham,\n\nIt's nice to meet you! Being a Chief AI Engineer is an exciting role. What kind of AI projects are you working on these days? \n\nI'm eager to learn more about your work and how I can be helpful.  \n\n"

In [9]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content='Your name is Preetham.  \n\nI remember that from our first interaction! 😊  \n\nDo you have any other questions for me?\n', response_metadata={'token_usage': {'completion_tokens': 31, 'prompt_tokens': 100, 'total_tokens': 131, 'completion_time': 0.061813892, 'prompt_time': 0.00696713, 'queue_time': None, 'total_time': 0.068781022}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-42815633-4a17-4d92-ad51-5116fc7fbfa1-0', usage_metadata={'input_tokens': 100, 'output_tokens': 31, 'total_tokens': 131})

In [10]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content


Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'As an AI, I have no memory of past conversations and do not know your name.\n\nPlease tell me your name! 😊\n\n'

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is sana")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


"Hi Sana, it's nice to meet you!  \n\nIs there anything I can help you with today? 😊\n"

In [12]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Sana!  I remember now. 😊  \n\nIs there anything else I can help you with?\n'

Prompt templates


Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ('system','You are a helpful assistant, try to answer all the questions as accurate as possible'),
        MessagesPlaceholder(variable_name = 'messages')

    ]
)

chain = prompt|model

In [14]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Preetham")]})

AIMessage(content="Hello Preetham, it's nice to meet you! How can I help you today? 😊  \n\n", response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 36, 'total_tokens': 61, 'completion_time': 0.050162339, 'prompt_time': 0.002970623, 'queue_time': None, 'total_time': 0.053132962}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e9f7f999-ec2f-489d-9787-11b13ea8ec48-0', usage_metadata={'input_tokens': 36, 'output_tokens': 25, 'total_tokens': 61})

In [15]:
with_message_history = RunnableWithMessageHistory(chain,get_session_history)

In [16]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is Jane")],
    config=config
)

response


Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


AIMessage(content="Hello Jane, it's nice to meet you! 😊 \n\nHow can I help you today?\n", response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 35, 'total_tokens': 59, 'completion_time': 0.048642479, 'prompt_time': 0.002610405, 'queue_time': None, 'total_time': 0.051252884000000005}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-89650ada-06e2-48b1-9d62-c62c94866833-0', usage_metadata={'input_tokens': 35, 'output_tokens': 24, 'total_tokens': 59})

In [17]:
response = with_message_history.invoke(
    [HumanMessage(content='What is my name')],
    config=config,
)
response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Jane.  I remember!  😄 \n\nIs there anything else I can help you with?\n'

In [18]:
#Add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),

        MessagesPlaceholder(variable_name='messages'),
    ]
)

chain = prompt| model

In [19]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is penny")],"language":"Telugu"})
response.content

'నమస్తే, పెన్నీ. 😊 \n\n(Namaste, Penny.) \n\nకిలాను మీకు సహాయం చేయగలను. మీకు ఏమి కావాలి? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history

In [20]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [21]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Krish")],"language":"Hindi"},
    config=config
)
repsonse.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'नमस्ते कृष्ण! 👋 \n\nआप कैसे हैं? 😊  \n'

In [22]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


In [23]:
response.content

'आपका नाम कृष्ण है. 😊  \n'

# Managing the Conversation History


One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in. 'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages.

In [24]:
from langchain_core.messages import SystemMessage, trim_messages
trimmer = trim_messages(
    max_tokens = 45,
    strategy = "last",
    token_counter = model,
    include_system = True,
    allow_partial = False,
    start_on = "human"
)

messages = [
    SystemMessage(content="You are a good assistant"),
    HumanMessage(content="I am bob"),
    AIMessage(content="hi"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),

]


In [27]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what\'s 2 + 2" 😊  \n'

In [28]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [29]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

Error in RootListenersTracer.on_chain_end callback: ValueError()
Error in callback coroutine: ValueError()


'Your name is Bob. 😊  \n\nI remember what you told me!\n'